In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import folium
import os

In [9]:

carpeta_co = "Datos_SIATA_Aire_co.json"

with open(carpeta_co, "r", encoding="utf-8") as f:data = json.load(f)

registros = []

        
for estacion in data:
    lat = estacion.get("latitud")
    lon = estacion.get("longitud")
    codigo = estacion.get("codigoSerial")
    nombre = estacion.get("nombre")
    nombreCorto = estacion.get("nombreCorto")
    datos = estacion.get("datos", [])

    for entrada in datos:
        registros.append({
            "codigoSerial": codigo,
            "nombreCorto": nombreCorto,
            "latitud": lat,
            "longitud": lon,
            "contaminante": entrada.get("variableConsulta"),
            "fecha": entrada.get("fecha"),
            "calibracion": entrada.get("valor"),
            "valor": entrada.get("calidad")
        })

# Convertimos a DataFrame como antes
df = pd.DataFrame(registros)
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
df["calibracion"] = pd.to_numeric(df["calibracion"], errors='coerce')


df.loc[df["calibracion"] == -9999, 'valor'] = np.nan

print(df.head(20))


    codigoSerial nombreCorto   latitud   longitud contaminante  \
0              3    GIR-SOSN  6.378433 -75.451141           co   
1              3    GIR-SOSN  6.378433 -75.451141           co   
2              3    GIR-SOSN  6.378433 -75.451141           co   
3              3    GIR-SOSN  6.378433 -75.451141           co   
4              3    GIR-SOSN  6.378433 -75.451141           co   
5              3    GIR-SOSN  6.378433 -75.451141           co   
6              3    GIR-SOSN  6.378433 -75.451141           co   
7              3    GIR-SOSN  6.378433 -75.451141           co   
8              3    GIR-SOSN  6.378433 -75.451141           co   
9              3    GIR-SOSN  6.378433 -75.451141           co   
10             3    GIR-SOSN  6.378433 -75.451141           co   
11             3    GIR-SOSN  6.378433 -75.451141           co   
12             3    GIR-SOSN  6.378433 -75.451141           co   
13             3    GIR-SOSN  6.378433 -75.451141           co   
14        

In [ ]:
validos_por_estacion = df.groupby('codigoSerial')['valor'].apply(lambda x: x.notna().mean() * 100)
validos_por_contaminante = df.groupby('contaminante')['valor'].apply(lambda x: x.notna().mean() * 100)


In [5]:
# Obtener contaminante único (ya que el archivo es solo de uno)
contaminante_unico = df['contaminante'].unique()[0] if not df['contaminante'].isna().all() else "Desconocido"
df

,codigoSerial,nombreCorto,latitud,longitud,contaminante,fecha,valor,calidad
0,3,GIR-SOSN,6.378433,-75.451141,co,2021-12-14 04:00:00,<NA>,4.31
1,3,GIR-SOSN,6.378433,-75.451141,co,2021-12-14 05:00:00,<NA>,4.31
2,3,GIR-SOSN,6.378433,-75.451141,co,2021-12-14 06:00:00,<NA>,4.31
3,3,GIR-SOSN,6.378433,-75.451141,co,2021-12-14 07:00:00,<NA>,4.31
4,3,GIR-SOSN,6.378433,-75.451141,co,2021-12-14 08:00:00,<NA>,4.31
...,...,...,...,...,...,...,...,...
26278,48,SUR-TRAF,6.152313,-75.627487,co,2022-12-14 00:00:00,<NA>,4.0
26279,48,SUR-TRAF,6.152313,-75.627487,co,2022-12-14 01:00:00,<NA>,4.0
26280,48,SUR-TRAF,6.152313,-75.627487,co,2022-12-14 02:00:00,<NA>,4.0
26281,48,SUR-TRAF,6.152313,-75.627487,co,2022-12-14 03:00:00,<NA>,4.0


In [7]:

df['mes'] = df['fecha'].dt.to_period("M")
promedios_mensuales = df.groupby('mes')['calidad'].mean().reset_index()
promedios_mensuales['mes'] = promedios_mensuales['mes'].astype(str)

# Graficar
plt.figure(figsize=(12,6))
sns.lineplot(data=promedios_mensuales, x='mes', y='calidad', marker='o')
plt.title(f'Promedios mensuales - {contaminante_unico.upper()}')
plt.xlabel('Mes')
plt.ylabel('Concentración')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
# 5. Mapa estático de estaciones
# ------------------------------

# Media por estacion
media_estaciones = calidad_aire_df.groupby(['codigoSerial', 'latitud', 'longitud'])['valor'].mean().reset_index()

mapa = folium.Map(location=[6.25, -75.6], zoom_start=11)

for _, row in media_estaciones.iterrows():
    folium.CircleMarker(
        location=[row['latitud'], row['longitud']],
        radius=6,
        popup=f"Estación {row['codigoSerial']}\nMedia: {row['valor']:.2f}",
        color='red' if row['valor'] > 25 else 'green',
        fill=True,
    ).add_to(mapa)

mapa